### 카메라 내부 캘리브레이션
- 갖고있는 데이터를 활용하여 캘리브레이션을 진행해보자.
- 보드: 차루코보드

In [1]:
import cv2
import numpy as np
import json
from glob import glob
from pathlib import Path
import sys
sys.path.append("/home/ros/llm_robot/")
from utils.camera import capture_d455_images, load_intrinsics
    

In [2]:
def get_charuco_board():
    # 1. ArUco 딕셔너리 정의
    aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_50)

    # 2. ChArUco 보드 생성
    charuco_board = cv2.aruco.CharucoBoard(
        size=(7, 5),
        squareLength=0.028,     # m
        markerLength=0.013,     # m
        dictionary=aruco_dict
    )
    return charuco_board

def read_gray_image(c_path):
    # 4. 이미지 로드 및 그레이스케일 변환 + CLAHE 적용
    #    조명 조건에 관계없이 안정적으로 코너를 검출하도록 대비 향상
    img = cv2.imread(c_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    gray = clahe.apply(gray)
    
    return gray


In [3]:
MAIN_DIR = "/home/ros/llm_robot/data/Calibration/Eye-to-Hand11"

intr_dir = sorted(glob(f"{MAIN_DIR}/intrinsics/*.json"))
c_dir = sorted(glob(f"{MAIN_DIR}/color/*.jpg"))

len(intr_dir), len(c_dir)

(25, 25)

In [6]:
camera_matrix, dist_coeffs = load_intrinsics(intr_dir[0])

In [7]:
allCorners = []
allIds = []
imgSize = None  

for c_path in c_dir:
    # 3. ChArUco 디텍터 생성
    charuco_board = get_charuco_board()
    detector = cv2.aruco.CharucoDetector(charuco_board)
    gray = read_gray_image(c_path)

    # 5. 내부 코너 및 마커 검출 (detectBoard)
    charuco_corners, charuco_ids, marker_corners, marker_ids = detector.detectBoard(gray)
    
    if charuco_corners is not None and charuco_ids is not None and len(charuco_corners) > 0:
        allCorners.append(charuco_corners)
        allIds.append(charuco_ids)
        imgSize = gray.shape[::-1]

In [8]:
ret, camera_matrix2, dist_coeffs2, rvecs, tvecs = cv2.aruco.calibrateCameraCharuco(
    allCorners, allIds,
    charuco_board, imgSize,
    camera_matrix, dist_coeffs)

In [9]:
camera_matrix, dist_coeffs

(array([[425.4894 ,   0.     , 427.1854 ],
        [  0.     , 424.91083, 245.99588],
        [  0.     ,   0.     ,   1.     ]], dtype=float32),
 array([-0.05594644,  0.06878078, -0.00011233,  0.00074334, -0.02200594],
       dtype=float32))

In [10]:
camera_matrix2, dist_coeffs2

(array([[420.75176178,   0.        , 429.06025809],
        [  0.        , 420.24855903, 244.88823229],
        [  0.        ,   0.        ,   1.        ]]),
 array([[-0.06766468],
        [ 0.07548154],
        [-0.00126559],
        [ 0.0005072 ],
        [-0.02807756]]))

In [11]:
fx = camera_matrix2[0,0]
fy = camera_matrix2[1,1]
ppx = camera_matrix2[0,2]
ppy = camera_matrix2[1,2]

In [12]:
# 결과 저장 (미터 단위)
camera_intrinsics_result = {
  "color_intrinsics": {
    "width": 848,
    "height": 480,
    "fx": fx,
    "fy": fy,
    "ppx": ppx,
    "ppy": ppy,
    "distortion_model": "distortion.inverse_brown_conrady",
    "distortion_coeffs": dist_coeffs2.flatten().tolist()
  }
}
camera_intrinsics_result

{'color_intrinsics': {'width': 848,
  'height': 480,
  'fx': np.float64(420.7517617775355),
  'fy': np.float64(420.24855903289927),
  'ppx': np.float64(429.0602580915742),
  'ppy': np.float64(244.8882322853532),
  'distortion_model': 'distortion.inverse_brown_conrady',
  'distortion_coeffs': [-0.06766467955296547,
   0.07548153843496587,
   -0.0012655901589318334,
   0.0005071996336020576,
   -0.02807756171443588]}}

In [13]:
result_path = f"{MAIN_DIR}/camera_intrinsics_result.json"
with open(result_path, 'w') as f:
    json.dump(camera_intrinsics_result, f, indent=2)

## 새롭게 정의한 calibration 결과로 재계산

In [12]:
# 저장된 데이터 로드
pose_dir = sorted(glob(f"{MAIN_DIR}/poses/*.json"))
charuco_dir = sorted(glob(f"{MAIN_DIR}/*.json"))[:25]
len(charuco_dir), len(pose_dir)

(25, 25)

In [13]:
T_base2ee_list = []  # 로봇 베이스 -> End-Effector (미터 단위)
T_cam2target_list = []  # 카메라 -> ChArUco 타겟 (미터 단위)
quality_weights = []  # 품질 점수를 가중치로 사용

print(f"총 {len(charuco_dir)}개의 포즈 데이터를 로드합니다...")

# 모든 charuco 파일을 사용 (ChArUco 코너 기반과 ArUco 마커 기반 모두 포함)
valid_charuco_files = []

for i, charuco_file in enumerate(charuco_dir):
    # ChArUco 포즈 로드 (타겟 -> 카메라) - Eye-to-Hand 구조
    with open(charuco_file, 'r') as f:
        charuco_data = json.load(f)
    
    # 품질 점수를 가중치로 사용
    quality_score = charuco_data.get("quality_score", 0.5)  # 기본값 0.5
    quality_weights.append(quality_score)
    
    # 새로운 형식에서는 target2cam 데이터를 사용
    if "rvec_target2cam" in charuco_data:
        rvec = np.array(charuco_data["rvec_target2cam"])
        tvec = np.array(charuco_data["tvec_target2cam"])
    else:
        # 기존 형식 호환성을 위해 역행렬 계산
        rvec_old = np.array(charuco_data["rvec"])
        tvec_old = np.array(charuco_data["tvec"])
        R_cam2target, _ = cv2.Rodrigues(rvec_old)
        R_target2cam = R_cam2target.T
        t_target2cam = -R_target2cam @ tvec_old
        rvec = cv2.Rodrigues(R_target2cam)[0]
        tvec = t_target2cam
    
    # 회전 벡터를 회전 행렬로 변환
    R_cam2target, _ = cv2.Rodrigues(rvec)
    
    # 카메라 -> 타겟 (미터 단위) - Eye-to-Hand 구조에 맞게 수정
    T_cam2target = np.eye(4)
    T_cam2target[:3, :3] = R_cam2target.T  # 역행렬
    T_cam2target[:3, 3] = -R_cam2target.T @ tvec.flatten()  # 역변환
    T_cam2target_list.append(T_cam2target)
    
    # 로봇 포즈 처리 (pose 파일이 있는 경우)
    if i < len(pose_dir) and pose_dir[i] is not None:
        with open(pose_dir[i], 'r') as f:
            pose_data = json.load(f)
        
        t_base2ee = np.array(pose_data["t_base2ee"])
        R_base2ee = np.array(pose_data["R_base2ee"])
    
    # 로봇 베이스 -> End-Effector (미터 단위)
    T_base2ee = np.eye(4)
    T_base2ee[:3, :3] = R_base2ee
    T_base2ee[:3, 3] = t_base2ee.flatten()
    T_base2ee_list.append(T_base2ee)
    
    # 디버그 정보
    detection_method = charuco_data.get("detection_method", "unknown")
    distance_mm = charuco_data.get("distance_mm", 0)
    print(f"포즈 {i+1}: {detection_method}, 거리={distance_mm:.1f}mm, 품질={quality_score:.3f}")

# 품질 점수 기반 가중치 정규화
quality_weights = np.array(quality_weights)
quality_weights = quality_weights / np.sum(quality_weights)  # 합이 1이 되도록 정규화

print(f"\n품질 점수 기반 가중치:")
for i, (weight, score) in enumerate(zip(quality_weights, quality_weights * np.sum(quality_weights))):
    print(f"  포즈 {i+1}: 품질={score:.3f}, 가중치={weight:.3f}")



총 25개의 포즈 데이터를 로드합니다...
포즈 1: charuco_corners, 거리=579.7mm, 품질=0.801
포즈 2: charuco_corners, 거리=509.8mm, 품질=0.359
포즈 3: charuco_corners, 거리=344.2mm, 품질=1.122
포즈 4: charuco_corners, 거리=369.6mm, 품질=0.399
포즈 5: charuco_corners, 거리=464.1mm, 품질=0.423
포즈 6: charuco_corners, 거리=614.9mm, 품질=0.738
포즈 7: charuco_corners, 거리=553.4mm, 품질=0.829
포즈 8: charuco_corners, 거리=336.5mm, 품질=1.093
포즈 9: charuco_corners, 거리=426.7mm, 품질=0.382
포즈 10: charuco_corners, 거리=345.8mm, 품질=0.963
포즈 11: charuco_corners, 거리=497.2mm, 품질=0.328
포즈 12: charuco_corners, 거리=396.8mm, 품질=1.163
포즈 13: charuco_corners, 거리=290.4mm, 품질=1.037
포즈 14: charuco_corners, 거리=373.2mm, 품질=1.029
포즈 15: charuco_corners, 거리=296.5mm, 품질=1.076
포즈 16: charuco_corners, 거리=446.6mm, 품질=0.418
포즈 17: charuco_corners, 거리=323.0mm, 품질=1.164
포즈 18: charuco_corners, 거리=382.1mm, 품질=1.142
포즈 19: charuco_corners, 거리=244.3mm, 품질=1.282
포즈 20: charuco_corners, 거리=428.2mm, 품질=0.360
포즈 21: charuco_corners, 거리=376.6mm, 품질=1.125
포즈 22: charuco_corners, 거리=349.8mm, 품질=1

In [14]:
# Eye-to-Hand calibration 수행 (가중치 포함)
print("\nEye-to-Hand calibration 계산 중...")
R_cam2base_list = []
t_cam2base_list = []

for i, (T_cam2target, T_base2ee) in enumerate(zip(T_cam2target_list, T_base2ee_list)):
    # T_cam2base = T_cam2target * T_base2ee.inverse()
    T_base2ee_inv = np.linalg.inv(T_base2ee)
    T_cam2base = T_cam2target @ T_base2ee_inv
    
    R_cam2base = T_cam2base[:3, :3]
    t_cam2base = T_cam2base[:3, 3]
    
    R_cam2base_list.append(R_cam2base)
    t_cam2base_list.append(t_cam2base)



Eye-to-Hand calibration 계산 중...


In [15]:
if R_cam2base is not None:
    # 결과는 미터 단위로 반환됨
    t_cam2base_norm = np.linalg.norm(t_cam2base)
    print(f"t_cam2base 크기 (m): {t_cam2base_norm:.3f}")
    
    # 미터 단위로 저장 (표준 단위)
    print(f"단위: 미터 (표준 단위)")
    
    # 변환 행렬 생성
    T_cam2base = np.eye(4)
    T_cam2base[:3, :3] = R_cam2base
    T_cam2base[:3, 3] = t_cam2base.flatten()
    
    # 결과 저장 (미터 단위)
    calibration_result = {
        "R_cam2base": R_cam2base.tolist(),
        "t_cam2base": t_cam2base.flatten().tolist(),  # 미터 단위로 저장
        "num_poses": len(T_base2ee_list),
        "description": "카메라 -> 로봇 베이스 변환 행렬 (Eye-to-Hand calibration, 미터 단위, 품질 가중치 적용)",
        "coordinate_transformation": {
            "applied": False,
            "description": "축보정 미적용 - 원본 카메라 좌표계 유지"
        },
        "quality_weighting": {
            "applied": True,
            "description": "품질 점수 기반 가중치 적용",
            "weights": quality_weights.tolist(),
            "quality_scores": (quality_weights * np.sum(quality_weights)).tolist()
        }
    }
    


t_cam2base 크기 (m): 0.568
단위: 미터 (표준 단위)


In [16]:
calibration_result

{'R_cam2base': [[0.01537349129400446,
   0.999876060574215,
   -0.0033940029496022056],
  [0.9901820721987494, -0.014752607420565604, 0.13900296568953274],
  [0.13893566734868304, -0.005497641756554601, -0.9902861486830423]],
 't_cam2base': [0.11749627956124493, -0.11307452232903055, 0.5436150456837687],
 'num_poses': 25,
 'description': '카메라 -> 로봇 베이스 변환 행렬 (Eye-to-Hand calibration, 미터 단위, 품질 가중치 적용)',
 'coordinate_transformation': {'applied': False,
  'description': '축보정 미적용 - 원본 카메라 좌표계 유지'},
 'quality_weighting': {'applied': True,
  'description': '품질 점수 기반 가중치 적용',
  'weights': [0.03878858110019707,
   0.017393150512256404,
   0.054337339230141224,
   0.019330329023867433,
   0.020455039517460916,
   0.035731402239442024,
   0.04015702022070649,
   0.05290544182717898,
   0.018513709311729968,
   0.046631036292397024,
   0.015886561685427094,
   0.05632195608584499,
   0.050181412000155974,
   0.04982087645185263,
   0.05210830324865061,
   0.020214553466307367,
   0.0563554754578

In [17]:
result_path = f"{MAIN_DIR}/cam2base2.json"
with open(result_path, 'w') as f:
    json.dump(calibration_result, f, indent=2)

# numpy 배열을 안전하게 float로 변환
t_x = float(t_cam2base.flatten()[0])
t_y = float(t_cam2base.flatten()[1])
t_z = float(t_cam2base.flatten()[2])

print(f"✅ Eye-to-Hand 캘리브레이션 완료!")
print(f"결과 저장: {result_path}")
print(f"카메라 -> 로봇 베이스 변환 행렬:")
print(f"R:\n{R_cam2base}")
print(f"t (m): [{t_x:.3f}, {t_y:.3f}, {t_z:.3f}]")
print(f"품질 가중치 적용됨: {len(quality_weights)}개 포즈")

✅ Eye-to-Hand 캘리브레이션 완료!
결과 저장: /home/ros/llm_robot/data/Calibration/Eye-to-Hand10/cam2base2.json
카메라 -> 로봇 베이스 변환 행렬:
R:
[[ 0.01537349  0.99987606 -0.003394  ]
 [ 0.99018207 -0.01475261  0.13900297]
 [ 0.13893567 -0.00549764 -0.99028615]]
t (m): [0.117, -0.113, 0.544]
품질 가중치 적용됨: 25개 포즈
